In [1]:
from queue import PriorityQueue, LifoQueue
import numpy as np
import random as rnd
import copy

In [2]:
class Cell: 
    def __init__(self, row, col, prev = None):
        self.row = row
        self.col = col
        self.prev = prev
    def __lt__(self, other):        
        if (self.row == other.row):
            return self.col < other.col
        return self.row < other.row
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col and self.prev is other.prev
    def __str__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")
    def __repr__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")

In [3]:
class Maze:
    # -1 is barrier, 0 is free
    def __init__(self, dim, p, q = 0):
        self.board =  np.zeros([dim, dim], dtype=int)
        self.dim = dim
        self.currentfire = []
        self.q = q
        self.p = p
        obstaclesList = []
        self.board_prob = {}
        self.firetuples = []
        self.timestep = 0
        for i in range(0, dim): #range() generates a list from 0 to dim-1 which can take up a lot of memory
            for j in range(0, dim):             
                if(rnd.random() < p):
                    self.board[i, j] = -1
                    obstaclesList.append((i, j))
        self.obstacles = np.array(obstaclesList)
        self.board[0, 0] = 0
        self.board[dim - 1, dim - 1] = 0
        
    def isSolvable(self):
        return pathExists(self)
    
    def startFire(self):
        i = rnd.randint(0,self.dim-1)
        j = rnd.randint(0,self.dim-1)
        while(self.board[i,j] == -1 or (i==0 and j == 0) or (i == self.dim-1 and j == self.dim-1)):
            i = rnd.randint(0,self.dim-1)
            j = rnd.randint(0,self.dim-1)
        self.board[i,j] = 2
        self.currentfire.append([i,j,None])
        self.firetuples.append((i,j)) # change for simulation
        
    def iterateFire(self):
        visited = [] #neighbors already visited in current iteration
        nowonfire = [] #need to append all new fires at the end 
        newFires = [] # change for simulation
        for f in self.currentfire:
            if(f[2] != 0): #check to see if all neighbors are already on fire. 
                neighbors = findAllNeighbors(f[0:2],self) #calculate valid neighbors of f, the cell that is currently on fire
                ofnbor = 0 #counter to find out how many neighbors are on fire
                for n in neighbors: #loop through the neighbors of f
                    nr,nc = n 
                    if (self.board[nr,nc] == 2): 
                        ofnbor = ofnbor + 1 
                    else:
                        if(n[0:2] not in visited): #only roll once for each cell not on fire
                            if(self.board[nr,nc] != -1):
                                surroundings = findAllNeighbors(n[0:2],self) #see what the neighbors are for the neighbor of f
                                k = 0 #k will be guaranteed to be at least 1
                                for nn in surroundings:
                                    nnr,nnc = nn
                                    if(self.board[nnr,nnc] == 2):
                                        k = k + 1 #counter
                                if(rnd.random() <= (1-((1-self.q)**k))): #roll
                                    self.board[nr,nc] = 2 
                                    nowonfire.append([nr,nc,None])
                                    newFires.append((nr,nc)) # change for simulation
                                    ofnbor = ofnbor + 1
                                else:
                                    visited.append([nr,nc])
                if(ofnbor == len(neighbors)):
                        f[2] = 0
        if(newFires): # change for simulation
            for y in newFires:
                self.firetuples.append(y) #
        if(nowonfire):
            for x in nowonfire:
                self.currentfire.append(x)
            return True
        return False

In [4]:
def pathExists(maze, fromCoords = (0,0), toCoords = None):
    if toCoords is None:
        toCoords = (maze.dim - 1, maze.dim - 1)
    toRow, toCol = toCoords
    visited = set()
    fringe = LifoQueue()
    fringe.put(fromCoords)
    while not fringe.empty():
        currentCoords = fringe.get()
        currentRow, currentCol = currentCoords
        if (currentRow == toRow and currentCol == toCol):
            return True
        else:
            neighbors = findNeighboringCoords(currentCoords, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                fringe.put(neighbor)
            visited.add(currentCoords)
    return False

In [5]:
def findNeighboringCoords(coords, maze):
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != 0):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

def findNeighboringCells(cell, maze):
    neighboringCoords = findNeighboringCoords((cell.row, cell.col), maze)
    neighbors = []
    for neighbor in neighboringCoords:
        row, col = neighbor
        neighbors.append(Cell(row, col, cell))
    return neighbors

def findAllNeighbors(coords,maze): 
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] == -1):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

In [6]:
def uniformCost(maze, cell):
    return 1
def adjacentFireHeuristic(maze, cell):
    cellRow = cell.row
    cellCol = cell.col
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    weight = 1
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != 2):
            continue
        weight += 1
    return weight
    
def shortestPathSearch(maze, startCoords = (0, 0), heuristicFunction = uniformCost, findNeighbors = findNeighboringCells):
    startRow, startCol = startCoords
    startCell = Cell(startRow, startCol)
    visited = set()
    fringe = PriorityQueue()
    fringe.put((0, startCell))
    while not fringe.empty():
        pathLength, currentCell = fringe.get()
        if (currentCell.row == maze.dim - 1 and currentCell.col == maze.dim - 1):
            shortestPath = []
            while (currentCell != startCell):
                shortestPath.append((currentCell.row, currentCell.col))
                currentCell = currentCell.prev
            shortestPath.append((startRow, startCol))
            shortestPath.reverse()
            return shortestPath
        else:
            neighbors = findNeighbors(cell = currentCell, maze = maze)
            for neighbor in neighbors:
                if (neighbor.row, neighbor.col) in visited:
                    continue
                nextPathLength = pathLength + heuristicFunction(maze = maze, cell = neighbor)
                fringe.put((nextPathLength, neighbor))
            visited.add((currentCell.row, currentCell.col))
    return None

In [7]:
def run_secondStrat(maze):
    """
    Algorithm:
    1. Identify shortest path to goal
        1.1. If found, move agent according to path
        1.2. Else, keep the agent in place
    2. Check if the agent reached the goal
        2.1. If true, terminate and return success
        2.2. Else, continue
    3. Advance the fire and check if the agent is burned
        3.1. If burned, terminate and return failure
        3.2. Else, continue at step 1
    """
    m = maze
    m.startFire()
    agent = (0,0)
    while True: #Loop exit conditions: Agent reaches goal or agent gets burned
        shortestPath = shortestPathSearch(m, agent) #recalculate the shortest path from the agent to the goal at agent's turn
        if shortestPath != None: #if a shortest path was found, move the agent in that direction
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1): #returns true if the agent reaches the goal, otherwise continue
            print("Agent Reached Goal")
            return True
        m.iterateFire() #advances the fire immediately after the agent's turn
        if m.board[agent] == 2: #returns false if the fire reaches the agent, otherwise continue
            print("Agent Burned")
            return False

In [8]:
def run_thirdStrat(maze):
    m = maze
    m.startFire()
    agent = (0,0)
    while True:
        shortestPath = shortestPathSearch(m, agent, adjacentFireHeuristic)
        
        print("Shortest Path")
        print(shortestPath)
        print("Agent Location")
        print(agent)
        print("Board")
        print(m.board)
        
        if shortestPath != None:
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1):
            print("Agent Reached Goal")
            return True
        m.iterateFire()
        if m.board[agent] == 2:
            print("Agent Burned")
            return False

In [9]:
# import copy
# def simulate(maze):
#     copiedMaze = copy.deepcopy(maze)
#     dim = maze.dim
#     #use self.currentfire to keep track of new fires
#     #generate 2d array of locations
#     i = 0
#     board_prob = {}
#     while (i < dim):
#         j = 0
#         while (j < dim):
#             board_prob[(i,j)] = np.zeros(dim**2)
#             j = j + 1
#         i = i + 1
# #     for o in copiedMaze.obstacles:
# #         del board_prob[o]
#     i = 0  
#     newFires = []
#     oldFires = []
#     num_sims = 30
#     for x in range(num_sims):
#         copiedMaze = copy.deepcopy(maze)
#         while(i < dim**2):
#             oldFires = copy.copy(copiedMaze.firetuples)
#             print("old fire:")
#             print(oldFires)
#             if(copiedMaze.iterateFire()): # if there are still new fires added
# #                 newFires = np.setdiff1d(newFires, oldFires)
#                 newFires = set(copiedMaze.firetuples) - set(oldFires) # do set difference to find which are actually the new fires
#                 print("NewFires:")
#                 print(newFires)
#                 for f in newFires: # for each of the new fires, increment the corresponding array at the index which corresponds to the timestep
#                     ary = copy.copy(board_prob[f[0],f[1]])
#                     ary[i] += 1                  
#                     board_prob[(f[0],f[1])] = ary 
#             else:
#                 i += 1
#                 continue
#                 #maybe loop thru the whole list and then check if its on fire. if it is then 1, else 0 
#                 #how to optimize: if 1, delete from this dictionary and add to another that isn't looped through. 
#             i = i + 1
#     for cll in board_prob: # update probability board to have the probabilites instead of counts
#         arry = board_prob[cll] 
#         probs = np.divide(arry, num_sims)
#         board_prob[cll] = probs
#     maze.board_prob = board_prob
# # Note: in the line with set difference, it won't work because of the third argument (i,j,__); __ being 3rd argument     

In [10]:
def simulationHeuristic(maze,cell):
    cellRow = cell.row
    cellCol = cell.col
    probArray = maze.board_prob[(cellRow,cellCol)]
    weight = 1
    weight += probArray[maze.timestep]
    return weight

In [11]:
def run_strat4(maze):
    simulate(maze) #probability table
    agent = (0,0)
    
    while True:
        shortestPath = shortestPathSearch(maze, agent, simulationHeuristic)
        print(shortestPath)
        print(maze.board)
        if shortestPath != None:
            agent = shortestPath[1]
            maze.timestep += 1
        if agent == (maze.dim-1,maze.dim-1):
            return True
        maze.iterateFire()
        if maze.board[agent] == 2:
            return False

In [15]:
import copy
def simulate(maze):
    copiedMaze = copy.deepcopy(maze) # copiedMaze is copy of maze with initial fire
    i = 0
    while (i < maze.dim): # initialize each cell to list of 0s
        j = 0
        while (j < maze.dim):
            maze.board_prob[(i,j)] = np.zeros(maze.dim**2)
            j = j + 1
        i = i + 1

    i = 0
    newFires = [] #
    oldFires = [] #
    num_sims = 100
    
    for x in range(num_sims):
        #print(maze.board_prob)
        copiedMaze = copy.deepcopy(maze)
#         copiedMaze.board = copy.copy(maze.board)
#         copiedMaze.currentfire = copy.copy(maze.currentfire)
#         copiedMaze.firetuples = copy.copy(maze.firetuples)
        i=0
        oldFires = []
        while (i < maze.dim**2):
            oldFires = copy.deepcopy(copiedMaze.firetuples) #
            if(copiedMaze.iterateFire()):
                #newFires = np.setdiff1d(copiedMaze.firetuples, oldFires) #
                newFires = list(set(copiedMaze.firetuples) - set(oldFires)) #
                #print(copiedMaze.board)
                #print(oldFires)
#                 print(newFires)
                for f in newFires:
                    maze.board_prob[(f[0],f[1])][i] += 1
            i+=1
    for cll in maze.board_prob: # update probability board to have the probabilites instead of counts
        arry = maze.board_prob[cll] 
        probs = np.divide(arry, num_sims)
        maze.board_prob[cll] = probs

In [25]:
m = Maze(10, 0.2,0.5)
while(m.isSolvable() == False):
    m = Maze(10, 0.2,0.5)
m.startFire()
print(run_strat4(m))
#print(m.board)
#print(m.board_prob)

[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (8, 8), (8, 9), (9, 9)]
[[ 0 -1 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0 -1  0 -1  0  0 -1]
 [ 0  0  0  0  0  0  2 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0  0  0]
 [ 0 -1  0  0  0 -1  0  0 -1 -1]
 [ 0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0  0  0]]
[(1, 0), (1, 1), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (8, 4), (8, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]
[[ 0 -1 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0 -1  0 -1  0  0 -1]
 [ 0  0  0  0  0  2  2 -1  0  0]
 [ 0  0  0  0  0  0  2  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0  0  0]
 [ 0 -1  0  0  0 -1  0  0 -1 -1]
 [ 0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0  0  0]]
[(1, 1), (2, 1), (2, 2), (3, 2), (4, 2),